
Obervations and Insights: The Power of Plots

In [117]:
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

In [118]:
#Bring in csv files and put them into dataframes
mouse_metadata_path = "Resources/Mouse_metadata.csv"
study_results_path = "Resources/Study_results.csv"
mouse_metadata_df = pd.read_csv(mouse_metadata_path)
study_results_df = pd.read_csv(study_results_path)


In [119]:
#Combine the data into a single dataset
merged_df = pd.merge(mouse_metadata_df,study_results_df, on="Mouse ID")
merged_df


,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
0,k403,Ramicane,Male,21,16,0,45.000000,0
1,k403,Ramicane,Male,21,16,5,38.825898,0
2,k403,Ramicane,Male,21,16,10,35.014271,1
3,k403,Ramicane,Male,21,16,15,34.223992,1
4,k403,Ramicane,Male,21,16,20,32.997729,1
...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2
1889,z969,Naftisol,Male,9,30,30,65.841013,3
1890,z969,Naftisol,Male,9,30,35,69.176246,4
1891,z969,Naftisol,Male,9,30,40,70.314904,4


In [120]:
#Get Unique number of Mouse by Mouse ID
uniquemouse = merged_df['Mouse ID'].nunique()
print(uniquemouse)

249


In [121]:
#Maximum Timecount is 10 per Mouse ID.  If Mouse ID has greater than 10, delete.
mousetp_count = merged_df[["Mouse ID","Timepoint"]].groupby(['Mouse ID']).agg(['count'])
mousetp_count

,Timepoint
,count
Mouse ID,
a203,10
a251,10
a262,10
a275,10
a366,7
...,...
z435,3
z578,10


In [122]:
#Merge Total Time Count into dataframe to exclude TimeCounts > 10
cleanmerged_df = pd.merge(merged_df,mousetp_count, on="Mouse ID")
cleanmerged_df
cleanmerged_df.rename(columns={cleanmerged_df.columns[8]: "sumtimecount"}, inplace = True)
cleanmerged_df



,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites,sumtimecount
0,k403,Ramicane,Male,21,16,0,45.000000,0,10
1,k403,Ramicane,Male,21,16,5,38.825898,0,10
2,k403,Ramicane,Male,21,16,10,35.014271,1,10
3,k403,Ramicane,Male,21,16,15,34.223992,1,10
4,k403,Ramicane,Male,21,16,20,32.997729,1,10
...,...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2,10
1889,z969,Naftisol,Male,9,30,30,65.841013,3,10
1890,z969,Naftisol,Male,9,30,35,69.176246,4,10
1891,z969,Naftisol,Male,9,30,40,70.314904,4,10


In [123]:

#Delete all Mouse ID entries with total timepoints > 10 "G989" has 13
cleanmerged_df=cleanmerged_df[cleanmerged_df['sumtimecount'] < 11]
cleanmerged_df


,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites,sumtimecount
0,k403,Ramicane,Male,21,16,0,45.000000,0,10
1,k403,Ramicane,Male,21,16,5,38.825898,0,10
2,k403,Ramicane,Male,21,16,10,35.014271,1,10
3,k403,Ramicane,Male,21,16,15,34.223992,1,10
4,k403,Ramicane,Male,21,16,20,32.997729,1,10
...,...,...,...,...,...,...,...,...,...
1888,z969,Naftisol,Male,9,30,25,63.145652,2,10
1889,z969,Naftisol,Male,9,30,30,65.841013,3,10
1890,z969,Naftisol,Male,9,30,35,69.176246,4,10
1891,z969,Naftisol,Male,9,30,40,70.314904,4,10


In [124]:
#Generate Summary Statistics table of the Tumor Volume for each drug regimen
#Set up calculations Mean, Median, Variance, Standard Deviation, and SEM
meantumorvol = cleanmerged_df.groupby('Drug Regimen')['Tumor Volume (mm3)'].mean()
mediantumorvol = cleanmerged_df.groupby('Drug Regimen')['Tumor Volume (mm3)'].median()
variancetumorvol = cleanmerged_df.groupby('Drug Regimen')['Tumor Volume (mm3)'].var()
stdevtumorvol = cleanmerged_df.groupby('Drug Regimen')['Tumor Volume (mm3)'].std()
semtumorvol = cleanmerged_df.groupby('Drug Regimen')['Tumor Volume (mm3)'].sem()
tumorvoldrug_df = pd.DataFrame({"Mean": meantumorvol, "Median": mediantumorvol, "Variance": variancetumorvol, "St Dev": stdevtumorvol, "SEM": semtumorvol})
tumorvoldrug_df

,Mean,Median,Variance,St Dev,SEM
Drug Regimen,,,,,
Capomulin,40.675741,41.557809,24.947764,4.994774,0.329346
Ceftamin,52.591172,51.776157,39.290177,6.268188,0.469821
Infubinol,52.884795,51.820584,43.128684,6.567243,0.492236
Ketapril,55.235638,53.698743,68.553577,8.279709,0.603860
Naftisol,54.331565,52.509285,66.173479,8.134708,0.596466
Placebo,54.033581,52.288934,61.168083,7.821003,0.581331
Propriva,52.320930,50.446266,43.852013,6.622085,0.544332
Ramicane,40.216745,40.673236,23.486704,4.846308,0.320955
Stelasyn,54.233149,52.431737,59.450562,7.710419,0.573111


In [135]:
#Generate Bar Plot by DataFrame.plot() Total Mice fore each treatment regimen
#Need the unique number of Mice IDs per regimen
mouseperdrug = cleanmerged_df.drop_duplicates(subset =['Mouse ID'])
mouseperdrug.sort_values(by=['Drug Regimen', 'Mouse ID'])
mouseperdrug



,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites,sumtimecount
0,k403,Ramicane,Male,21,16,0,45.0,0,10
10,s185,Capomulin,Female,3,17,0,45.0,0,10
20,x401,Capomulin,Female,16,15,0,45.0,0,10
30,m601,Capomulin,Male,22,17,0,45.0,0,10
40,g791,Ramicane,Male,11,16,0,45.0,0,10
...,...,...,...,...,...,...,...,...,...
1858,z314,Stelasyn,Female,21,28,0,45.0,0,2
1860,z435,Propriva,Female,12,26,0,45.0,0,3
1863,z581,Infubinol,Female,24,25,0,45.0,0,10
1873,z795,Naftisol,Female,13,29,0,45.0,0,10
